[View in Colaboratory](https://colab.research.google.com/github/hhngan/ML/blob/master/StockAnalyst/StockAnalyst.ipynb)

In [0]:
#Install libraries

!pip install -q botocore
!pip install -q certifi
!pip install -q colorama
!pip install -q cycler
!pip install -q docutils
!pip install -q jmespath
!pip install -q kiwisolver
!pip install -q matplotlib
!pip install -q numpy
!pip install -q pandas
!pip install -q pyasn1
!pip install -q pyparsing
!pip install -q python-dateutil
!pip install -q pytz
!pip install -q PyYAML
!pip install -q rsa
!pip install -q s3transfer
!pip install -q scikit-learn
!pip install -q six

# Buil Features

In [0]:
import pandas as pd
import numpy as np

produce_charts = False
if produce_charts:
  import matplotlib.pyplot as plt

def return_features(df):
  """
  Args:
    df: pandas.DataFrame, columns include at least ["date", "open", "high", "low", "close", "volume"]
  Returns:
    pandas.DataFrame
  """
  df["return"] = df["close"] / df["close"].shift(1)
  df["close_to_open"] = df["close"] / df["open"]
  df["close_to_high"] = df["close"] / df["high"]
  df["close_to_low"] = df["close"] / df["low"]
  df = df.iloc[1:] #first first row: does not have return a value
  return df

def target_value(df):
  df["y"] = df["return"].shift(-1)
  df = df.iloc[:len(df)-1]
  return df

def trend_features(df):
  """
  Args:
      df: pandas.DataFrame, columns include at least ["date", "open", "high", "low", "close", "volume"]
  Returns:
      pandas.DataFrame
  """
  df = macd(df)
  df = ma(df)
  df = parabolic_sar(df)
  return df

def macd(df):
  """
  Math reference: https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:moving_average_convergence_divergence_macd
  Args:
      df: pandas.DataFrame, columns include at least ["close"]
  Returns:
      pandas.DataFrame
  """
  ema_12_day = df["close"].ewm(com=(12-1)/2).mean()
  ema_26_day = df["close"].ewm(com=(26-1)/2).mean()
  df["macd_line"] = ema_12_day - ema_26_day
  df["macd_9_day"] = df["macd_line"].ewm(com=(9-1)/2).mean()
  df["macd_diff"] = df["macd_line"] - df["macd_9_day"]
  # print(df.tail(10)[["date", "close", "macd_line", "macd_9_day"]])
  if produce_charts:
    chart_macd(df)
  return df

def chart_macd(df):
  """
  Save chart to charts/macd
  Args:
      df: pandas.DataFrame, columns include at least ["date", "close", "macd_line", "macd_9_day"]
  Returns:
      None
  """
  fig, axes = plt.subplots(2, 1, figsize=(10, 10))
  fig.tight_layout()
  plt.suptitle("MSFT", fontsize=24)
  plt.subplots_adjust(left=0.1, top=0.9, hspace = 0.4)
  ax1 = axes[0]
  ax1.set_title("Price")
  ax1.set_ylabel("$")
  df.tail(300)[["date", "close"]].plot(x="date", kind="line", ax=ax1)
  ax2 = axes[1]
  ax2.set_title("MACD")
  df.tail(300)[["date", "macd_line", "macd_9_day"]].plot(x="date", kind="line", ax=ax2, secondary_y=False)
  # df.tail(300)[["date", "macd_diff"]].plot(x="date", kind="bar", ax=ax2, secondary_y=True)
  fig.savefig("charts/macd.png")
  
def ma(df):
  """
  Math reference: https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:moving_averages
  Args:
      df: pandas.DataFrame, columns include at least ["close"]
  Returns:
      pandas.DataFrame
  """
  df["ma_50_day"] = df["close"].rolling(50).mean()
  df["ma_200_day"] = df["close"].rolling(200).mean()
  df["ma_50_200"] = df["ma_50_day"] - df["ma_200_day"]
  # print(df.tail(10)[["date", "close", "ma_50_200"]])
  if produce_charts:
    chart_ma(df)
  return df

def chart_ma(df):
  """
  Save chart to charts/ma
  Args:
      df: pandas.DataFrame, columns include at least ["date", "close", "ma_50_200"]
  Returns:
      None
  """
  fig, axes = plt.subplots(1, 1, figsize=(10, 10))
  fig.tight_layout()
  plt.suptitle("MSFT Moving Average 50 day - 200 day", fontsize=24)
  plt.subplots_adjust(left=0.1, top=0.9, right=0.9, hspace=0.4)
  axes.set_ylabel("$")
  df.tail(1500)[["date", "close"]].plot(x="date", kind="line", ax=axes, secondary_y=False)
  df.tail(1500)[["date", "ma_50_day"]].plot(x="date", kind="line", ax=axes, secondary_y=False)
  df.tail(1500)[["date", "ma_200_day"]].plot(x="date", kind="line", ax=axes, secondary_y=False)
  df.tail(1500)[["date", "ma_50_200"]].plot(x="date", kind="line", ax=axes, secondary_y=True)
  fig.savefig("charts/ma.png")

def parabolic_sar(df):
  """
  Math reference: https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:parabolic_sar
  Args:
      df: pandas.DataFrame, columns include at least ["close"]
  Returns:
      pandas.DataFrame
  """
  df["sar"] = np.nan
  step = 5
  acc_factor = 0.02
  uptrend = False
  prior_sar = max(df.loc[1:step, "close"])
  extreme_point = min(df.loc[1:step, "close"])
  for i, row in df.iloc[step:].iterrows():
      if uptrend:
          df.at[i, "sar"] = prior_sar + acc_factor*(extreme_point - prior_sar)
          if df.at[i, "low"] < df.at[i, "sar"]:
              # reverse to downtrend
              uptrend = False
              prior_sar = max(df.loc[i-step:i, "close"])
              extreme_point = min(df.loc[i-step:i, "close"])
          else:
              # continue uptrend
              if df.at[i, "close"] > extreme_point:
                  extreme_point = df.at[i, "close"]
                  acc_factor = min(0.2, acc_factor+0.02)
      else:
          df.at[i, "sar"] = prior_sar - acc_factor*(prior_sar - extreme_point)
          if df.at[i, "high"] > df.at[i, "sar"]:
              # reverse to uptrend
              uptrend = True
              prior_sar = min(df.loc[i-step:i, "close"])
              extreme_point = max(df.loc[i-step:i, "close"])
          else:
              # continue downtrend
              if df.at[i, "close"] < extreme_point:
                  extreme_point = df.at[i, "close"]
                  acc_factor = min(0.2, acc_factor+0.02)
      prior_sar = df.at[i, "sar"]
  if produce_charts:
      chart_sar(df)
  return df

def chart_sar(df):
    """
    Save chart to charts/ma
    Args:
        df: pandas.DataFrame, columns include at least ["date", "close", "sar"]
    Returns:
        None
    """
    fig, axes = plt.subplots(1, 1, figsize=(10, 10))
    fig.tight_layout()
    plt.suptitle("MSFT SAR", fontsize=24)
    plt.subplots_adjust(left=0.1, top=0.9, right=0.9, hspace = 0.4)
    axes.set_ylabel("$")
    df.tail(100)[["date", "close"]].plot(x="date", kind="line", ax=axes, secondary_y=False)
    df.tail(100)[["date", "sar"]].plot(x="date", style=".", ax=axes, secondary_y=False)
    fig.savefig("charts/sar.png")

def momentum_features(df):
    """
    Args:
        df: pandas.DataFrame, columns include at least ["date", "open", "high", "low", "close", "volume"]
    Returns:
        pandas.DataFrame
    """
    df = stochastic_oscillator(df)
    df = commodity_channel_index(df)
    df = rsi(df)
    return df

def stochastic_oscillator(df):
    """
    Math reference: https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:stochastic_oscillator_fast_slow_and_full
    Args:
        df: pandas.DataFrame, columns include at least ["close"]
    Returns:
        pandas.DataFrame
    """
    lookback = 14
    df["stochastic_oscillator"] = ((df["close"] - df["close"].rolling(lookback).min()) /
        (df["close"].rolling(lookback).max() - df["close"].rolling(lookback).min())) * 100
    if produce_charts:
        chart_stochastic_oscillator(df)
    return df

def chart_stochastic_oscillator(df):
    """
    Save chart to charts/stochastic_oscillator
    Args:
        df: pandas.DataFrame, columns include at least ["date", "close", "stochastic_oscillator"]
    Returns:
        None
    """
    fig, axes = plt.subplots(1, 1, figsize=(10, 10))
    fig.tight_layout()
    plt.suptitle("MSFT Stochastic Oscillator", fontsize=24)
    plt.subplots_adjust(left=0.1, top=0.9, right=0.9, hspace = 0.4)
    df.tail(100)[["date", "close"]].plot(x="date", kind="line", ax=axes, secondary_y=False)
    df.tail(100)[["date", "stochastic_oscillator"]].plot(x="date", kind="line", ax=axes, secondary_y=True)
    fig.savefig("charts/stochastic_oscillator.png")

def commodity_channel_index(df):
    """
    Math reference: https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:commodity_channel_index_cci
    Args:
        df: pandas.DataFrame, columns include at least ["open", "high", "low", "close"]
    Returns:
        pandas.DataFrame
    """
    typical_price = (df["high"] + df["low"] + df["close"]) / 3
    mean_dev = abs(typical_price - typical_price.rolling(20).mean()).rolling(20).mean()
    df["cci"] = (typical_price - typical_price.rolling(20).mean()) / (0.15 * mean_dev)
    if produce_charts:
        chart_commodity_channel_index(df)
    return df

def chart_commodity_channel_index(df):
    """
    Save chart to charts/commodity_channel_index
    Args:
        df: pandas.DataFrame, columns include at least ["date", "close", "stochastic_oscillator"]
    Returns:
        None
    """
    fig, axes = plt.subplots(1, 1, figsize=(10, 10))
    fig.tight_layout()
    plt.suptitle("MSFT Commodity Channel Index (CCI)", fontsize=24)
    plt.subplots_adjust(left=0.1, top=0.9, right=0.9, hspace = 0.4)
    df.tail(300)[["date", "close"]].plot(x="date", kind="line", ax=axes, secondary_y=False)
    df.tail(300)[["date", "cci"]].plot(x="date", kind="line", ax=axes, secondary_y=True)
    fig.savefig("charts/commodity_channel_index.png")

def rsi(df):
    """
    Math reference: https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:relative_strength_index_rsi
    Args:
        df: pandas.DataFrame, columns include at least ["close"]
    Returns:
        pandas.DataFrame
    """
    df["dollar_pnl"] = df["close"].shift(1) - df["close"]
    avg_gains = df["dollar_pnl"].iloc[:14][df["dollar_pnl"].iloc[:14] > 0].sum() / 14
    avg_losses = abs(df["dollar_pnl"].iloc[:14][df["dollar_pnl"].iloc[:14] < 0].sum()) / 14
    for i, row in df.iloc[14:].iterrows():
        if row["dollar_pnl"] > 0:
            avg_gains = (avg_gains * 13 + row["dollar_pnl"]) / 14
        else:
            avg_losses = (avg_losses * 13 + abs(row["dollar_pnl"])) / 14
        if avg_losses == 0:
            rs = 100
        else:
            rs = avg_gains / avg_losses
        df.loc[i, "rsi"] = 100 - 100 / (1 + rs)
    # print(df.tail(20)[["date", "close", "rsi"]])
    if produce_charts:
        chart_rsi(df)
    return df

def chart_rsi(df):
    """
    Save chart to charts/rsi
    Args:
        df: pandas.DataFrame, columns include at least ["date", "close", "rsi"]
    Returns:
        None
    """
    fig, axes = plt.subplots(1, 1, figsize=(10, 10))
    fig.tight_layout()
    plt.suptitle("MSFT Relative Strength Index (RSI)", fontsize=24)
    plt.subplots_adjust(left=0.1, top=0.9, right=0.9, hspace = 0.4)
    df.tail(100)[["date", "close"]].plot(x="date", kind="line", ax=axes, secondary_y=False)
    df.tail(100)[["date", "rsi"]].plot(x="date", kind="line", ax=axes, secondary_y=True)
    fig.savefig("charts/rsi.png")

def volatility_features(df):
    """
    Args:
        df: pandas.DataFrame, columns include at least ["date", "open", "high", "low", "close", "volume"]
    Returns:
        pandas.DataFrame
    """
    df["5d_volatility"] = df["return"].rolling(5).std()
    df["21d_volatility"] = df["return"].rolling(21).std()
    df["60d_volatility"] = df["return"].rolling(60).std()
    df = bollinger_bands(df)
    df = average_true_range(df)
    return df

def bollinger_bands(df):
    """
    Math reference: https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:bollinger_bands
    Args:
        df: pandas.DataFrame, columns include at least ["close"]
    Returns:
        pandas.DataFrame
    """
    df["bollinger"] = ((df["close"] - df["close"].rolling(21).mean()) / 
        2 * df["close"].rolling(21).std())
    if produce_charts:
        chart_bollinger(df)
    return df

def chart_bollinger(df):
    """
    Save chart to charts/bollinger
    Args:
        df: pandas.DataFrame, columns include at least ["date", "close", "bollinger"]
    Returns:
        None
    """
    fig, axes = plt.subplots(1, 1, figsize=(10, 10))
    fig.tight_layout()
    plt.suptitle("MSFT Bollinger Bands", fontsize=24)
    plt.subplots_adjust(left=0.1, top=0.9, right=0.9, hspace = 0.4)
    df.tail(100)[["date", "close"]].plot(x="date", kind="line", ax=axes, secondary_y=False)
    df.tail(100)[["date", "bollinger"]].plot(x="date", kind="line", ax=axes, secondary_y=True)
    fig.savefig("charts/bollinger.png")

def average_true_range(df):
    """
    Math reference: https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:average_true_range_atr
    Args:
        df: pandas.DataFrame, columns include at least ["high" ,"low, "close"]
    Returns:
        pandas.DataFrame
    """
    high_vs_low = df["high"] - df["low"]
    high_vs_prev_close = df["high"] - df["close"].shift(-1)
    low_vs_prev_close = df["low"] - df["close"].shift(-1)
    tr = high_vs_low.to_frame("high_vs_low")
    tr["high_vs_prev_close"] = high_vs_prev_close
    tr["low_vs_prev_close"] = low_vs_prev_close
    tr["tr"] = tr.max(axis=1)
    df["atr"] = tr["tr"].rolling(14).mean()
    if produce_charts:
        chart_average_true_range(df)
    return df

def chart_average_true_range(df):
    """
    Save chart to charts/average_true_range
    Args:
        df: pandas.DataFrame, columns include at least ["date", "close", "atr"]
    Returns:
        None
    """
    fig, axes = plt.subplots(1, 1, figsize=(10, 10))
    fig.tight_layout()
    plt.suptitle("MSFT Average True Range (ATR)", fontsize=24)
    plt.subplots_adjust(left=0.1, top=0.9, right=0.9, hspace = 0.4)
    df.tail(100)[["date", "close"]].plot(x="date", kind="line", ax=axes, secondary_y=False)
    df.tail(100)[["date", "atr"]].plot(x="date", kind="line", ax=axes, secondary_y=True)
    fig.savefig("charts/average_true_range.png")

def volume_features(df):
    """
    Args:
        df: pandas.DataFrame, columns include at least ["date", "open", "high", "low", "close", "volume"]
    Returns:
        pandas.DataFrame
    """
    df["volume_rolling"] = df["volume"] / df["volume"].shift(21)
    df = on_balance_volume(df)
    df = chaikin_oscillator(df)
    return df

def on_balance_volume(df):
    """
    Math reference: https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:on_balance_volume_obv
    Args:
        df: pandas.DataFrame, columns include at least ["close", "volume"]
    Returns:
        pandas.DataFrame
    """
    df["dollar_pnl"] = df["close"].shift(1) - df["close"]
    df["on_balance_volume"] = df["volume"]
    df["on_balance_volume"] = df.apply(lambda row: row.volume * -1 if row.dollar_pnl < 0 else row.volume, axis=1)
    df["on_balance_volume"] = df["on_balance_volume"].cumsum()
    # print(df.head(10)[["date", "close", "dollar_pnl", "volume", "on_balance_volume"]])
    if produce_charts:
        chart_on_balance_volume(df)
    return df

def chart_on_balance_volume(df):
    """
    Save chart to charts/on_balance_volume
    Args:
        df: pandas.DataFrame, columns include at least ["date", "close", "on_balance_volume"]
    Returns:
        None
    """
    fig, axes = plt.subplots(1, 1, figsize=(10, 10))
    fig.tight_layout()
    plt.suptitle("MSFT On Balance Volume (OBV)", fontsize=24)
    plt.subplots_adjust(left=0.1, top=0.9, right=0.9, hspace = 0.4)
    df.tail(100)[["date", "close"]].plot(x="date", kind="line", ax=axes, secondary_y=False)
    df.tail(100)[["date", "on_balance_volume"]].plot(x="date", kind="line", ax=axes, secondary_y=True)
    fig.savefig("charts/on_balance_volume.png")

def chaikin_oscillator(df):
    """
    Math reference: https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:chaikin_oscillator
    Args:
        df: pandas.DataFrame, columns include at least ["low", "high", "close", "volume"]
    Returns:
        pandas.DataFrame
    """
    money_flow_multiplier = ((df["close"] - df["low"]) / (df["high"] - df["low"])) / (df["high"] - df["low"])
    money_flow_volume = df["volume"] * money_flow_multiplier
    adl = money_flow_volume.cumsum()
    df["chaikin_oscillator"] = adl.ewm(com=(3-1)/2).mean() - adl.ewm(com=(10-1)/2).mean()
    # print(df.head(10)[["date", "close", "volume", "chaikin_oscillator"]])
    if produce_charts:
        chart_chaikin_oscillator(df)
    return df

def chart_chaikin_oscillator(df):
    """
    Save chart to charts/chaikin_oscillator
    Args:
        df: pandas.DataFrame, columns include at least ["date", "close", "chaikin_oscillator"]
    Returns:
        None
    """
    fig, axes = plt.subplots(1, 1, figsize=(10, 10))
    fig.tight_layout()
    plt.suptitle("MSFT Chaikin Oscillator", fontsize=24)
    plt.subplots_adjust(left=0.1, top=0.9, right=0.9, hspace = 0.4)
    df.tail(100)[["date", "close"]].plot(x="date", kind="line", ax=axes, secondary_y=False)
    df.tail(100)[["date", "chaikin_oscillator"]].plot(x="date", kind="line", ax=axes, secondary_y=True)
    fig.savefig("charts/chaikin_oscillator.png")

# DATA PREPROCESSING

In [0]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing as sklearn_preprocessing
from sklearn.decomposition import PCA

In [13]:
def preprocessing():
    filename = "https://github.com/hhngan/ML/raw/master/StockAnalyst/MSFT.csv"
    df = yahoo_finance_source_to_df(filename)
    df = return_features(df)
    df = target_value(df)
    # print(df.head(5)[["date", "close", "volume", "return", "y"]])
    df = clean_df(df)
    df = new_features(df)
    X, y = remove_unused_features(df)
    X = X.values # convert to np.ndarray for sklearn
    y = y.values # convert to np.ndarray for sklearn
    X = scaling(X)
    X = dimensionality_reduction(X)
    train_test_split = 7000
    X_learn = X[:train_test_split]
    y_learn = y[:train_test_split]
    X_test = X[train_test_split:]
    y_test = y[train_test_split:]
    assert len(X_learn) + len(X_test) == len(X)
    df_to_csv(df=pd.DataFrame(X_learn), filename="MSFT_X_learn.csv")
    df_to_csv(df=pd.DataFrame(y_learn), filename="MSFT_y_learn.csv")
    df_to_csv(df=pd.DataFrame(X_test), filename="MSFT_X_test.csv")
    df_to_csv(df=pd.DataFrame(y_test), filename="MSFT_y_test.csv")

def yahoo_finance_source_to_df(filename):
    """
    Lead Yahoo Finance csv and format to DataFrame
    Args:
        filename: string
    Returns:
        pandas.DataFrame
    """
    df = pd.read_csv(filename, header=0)
    df.drop(labels="Close", axis=1, inplace=True)
    df.columns = ["date", "open", "high", "low", "close", "volume"]
    df["date"] = pd.to_datetime(df["date"])
    return df

def clean_df(df):
    """
    Args:
        df: pandas.DataFrame
    Returns:
        pandas.DataFrame
    """
    df = missing_values(df)
    df = outliers(df)
    return df

def missing_values(df):
    """
    Replace missing values with latest available
    Args:
        df: pandas.DataFrame
    Returns:
        pandas.DataFrame
    """
    missing_values_count = df.isnull().sum()
    # print("Original data # missing values: {}".format(missing_values_count))
    if sum(missing_values_count) == 0:
        return df
    else:
        print("Ffill of missing values necessary")
        df = df.fillna(method = "ffill", axis=0).fillna("0")
        missing_values_count = df.isnull().sum()
        # print("After filling na # missing values: {}".format(missing_values_count))
        assert sum(missing_values_count) == 0
        return df

def outliers(df):
    """
    Analyze outliers of dataset
    Args:
        df: pandas.DataFrame
    Returns:
        pandas.DataFrame
    """
    df_outliers = df.loc[:,["date", "return", "close_to_open", "close_to_high", "close_to_low"]]
    column_to_analysts = "return"
    df_smallest = df_outliers.sort_values(by=column_to_analysts, ascending=True)
    df_largest = df_outliers.sort_values(by=column_to_analysts, ascending=False)
    # print(df_smallest.iloc[:5])
    # print(df_largest.iloc[:5])
    return df

def new_features(df):
    """
    Generate feature useful for learning
    Args:
        df: pandas.DataFrame
    Returns:
        pandas.DataFrame
    """
    df = trend_features(df)
    df = momentum_features(df)
    df = volatility_features(df)
    df = volume_features(df)
    return df

def remove_unused_features(df):
    """
    - skip rows with nan values due to feature extraction
    - extract only useful columns for learning
    Args:
        df: pandas.DataFrame
    Returns:
        pandas.DataFrame
    """
    X = df.loc[200:len(df)-1, ["return", "close_to_open", "close_to_high", "close_to_low",
        "macd_diff", "ma_50_200", "sar", "stochastic_oscillator",
        "cci", "rsi", "5d_volatility", "21d_volatility", "60d_volatility",
        "bollinger", "atr", "on_balance_volume", "chaikin_oscillator"]]
    y = df.loc[200:len(df)-1, ["y"]]
    return X, y

def scaling(df):
    """
    Scale all features to unit variance and 0 mean to optimize training
    Args:
        df: pandas.DataFrame
    Returns:
        numpy.ndarray
    """
    X = sklearn_preprocessing.scale(df, axis=0)
    assert sum(X.mean(axis=0)) > -0.00001 and sum(X.mean(axis=0)) < 0.00001 # zero mean
    assert sum(X.std(axis=0)) > 0.99999 and sum(X.mean(axis=0)) < 1.00001 # unit variance
    return X

def dimensionality_reduction(X):
    """
    Analyzed the usefulness of dimensionality reduction
    Args:
        numpy.ndarray
    Returns:
        numpy.ndarray
    """
    sk_model = PCA(n_components=10)
    sk_model.fit_transform(X)
    sk_model.explained = sk_model.explained_variance_ratio_.cumsum()
    # PCA not used due to a small number of original features
    return X

def df_to_csv(df, filename):
    """
    Save DataFrame to csv for later use
    Args:
        df: pandas.DataFrame
        filename: string
    Returns:
        None
    """
    df.to_csv(filename, index=False)

if __name__ == "__main__":
    preprocessing()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [15]:
!ls
from google.colab import files
files.download('MSFT_X_test.csv')
files.download('MSFT_y_test.csv')
files.download('MSFT_X_learn.csv')
files.download('MSFT_y_learn.csv')

datalab		  MSFT_X_test.csv   MSFT_y_test.csv
MSFT_X_learn.csv  MSFT_y_learn.csv  sample_data
